决策树算法

In [10]:
import numpy as np

class DecisionTree:
    # 计算熵
    def entropy(self, y):
        if len(y) == 0: # 如果没有样本，熵为0
            return 0

        class_counts = np.bincount(y) # 统计每个类别的样本数
        probabilities = class_counts / len(y) # 计算各类的概率

        return -np.sum(probabilities * np.log2(probabilities + 1e-9)) # 1e-9:小数，防止出现对0取对数

    # 计算信息增益
    def information_gain(self, y, left_indices, right_indices):
        parent_entropy = self.entropy(y)  # 计算父节点的熵

        # 如果某一侧没有样本，则信息增益为0
        if len(y[left_indices]) == 0 or len(y[right_indices]) == 0:
            return 0

        n = len(y)  # 总样本数
        n_left = len(y[left_indices])  # 左侧样本数
        n_right = len(y[right_indices])  # 右侧样本数

        # 计算子节点的熵
        child_entropy = (
            (n_left / n) * self.entropy(y[left_indices]) +
            (n_right / n) * self.entropy(y[right_indices])
        )

        # 信息增益为父节点熵减去子节点熵
        return parent_entropy - child_entropy

    # 决策树设置
    def __init__(self, min_samples_split=2, max_depth=None):
        self.min_samples_split = min_samples_split  # 最小样本分裂数
        self.max_depth = max_depth  # 最大深度
        self.tree = None  # 对决策树根节点进行初始化

    # 拟合模型
    def fit(self, X, y):
        self.tree = self.build_tree(X, y)

    # 创建决策树
    def build_tree(self, X, y, depth=0):
        # 创建树的节点
        num_samples, num_features = X.shape # num_samples表示样本的数量，num_features表示特征的数量
        unique_classes = np.unique(y) # np.unique函数找到标签数组y中所有唯一的类标签，这将返回一个数组

        # 如果样本数少于最小分裂数或只有一个类，则返回叶节点
        if num_samples < self.min_samples_split or len(unique_classes) == 1:
            return self._create_leaf(y)

        # 如果达到最大深度，则返回叶节点
        if self.max_depth is not None and depth >= self.max_depth:
            return self._create_leaf(y)

        # 获取最佳特征和分裂点
        best_feature, best_threshold = self._best_split(X, y, num_features)

        # 如果没有找到合适的分裂点，则返回叶节点
        if best_feature is None:
            return self._create_leaf(y)

        # 根据最佳特征和分裂点进行分裂
        left_indices = X[:, best_feature] < best_threshold # X[:, best_feature]：从特征矩阵X中选择当前最佳特征（best_feature）的所有样本。
        right_indices = X[:, best_feature] >= best_threshold

        left_subtree = self.build_tree(X[left_indices], y[left_indices], depth + 1) # 递归地构建左子树，并将其存储在left_subtree变量中。
        right_subtree = self.build_tree(X[right_indices], y[right_indices], depth + 1)

        # 返回当前节点的信息
        return {
            'feature_index': best_feature,
            'threshold': best_threshold,
            'left': left_subtree,
            'right': right_subtree
        }

    # 寻找最佳特征和分裂点
    def _best_split(self, X, y, num_features):
        best_gain = -1  # 初始化最佳增益
        best_feature = None
        best_threshold = None

        for feature_index in range(num_features):
            unique_classes = np.bincount(y)
            thresholds, classes = zip(*sorted(zip(X[:, feature_index], y)))
            # 将当前特征的值（X[:, feature_index]）和对应的标签（y）配对，生成一个元组列表。每个元组的形式为(特征值, 标签)。
            # sorted(...)：将配对的元组按特征值进行排序，这样有助于找到潜在的分裂点。
            # zip(*)：解压排序后的元组列表，将特征值和标签分开。thresholds将包含特征值，classes将包含对应的标签。

            num_left = [0] * len(np.unique(y))
            num_right = [np.sum(classes == c) for c in unique_classes]

            # 初始化一个列表num_left，用于记录左子树中每个类的样本数量。np.unique(y)返回标签的唯一值，因此列表的长度等于类别的数量。初始时，每个类别的样本数量都设置为0。
            # 对于num_right生成一个列表，记录每个类出现的次数
            for i in range(1, len(classes)):  # 从第一个分裂点开始
                class_label = classes[i - 1]
                num_left[class_label] += 1
                num_right[class_label] -= 1

                # 计算当前分裂点的增益
                if thresholds[i] == thresholds[i - 1]:
                    continue  # 避免重复分裂

                gain = self.information_gain(y, num_left, num_right)

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature_index
                    best_threshold = (thresholds[i] + thresholds[i - 1]) / 2  # 取平均作为分裂点

            # 检查是否找到有效的分裂
            if best_gain > 0:
                return best_feature, best_threshold
            else:
                return None, None  # 返回无效的分裂标志

    # 返回出现最多的类作为叶节点
    def _create_leaf(self, y):
        return np.bincount(y).argmax()  # .argmax()：这个方法返回数组中最大值的索引

    # 预测函数
    def predict(self, X):
        return [self._predict(sample, self.tree) for sample in X]

    # 递归预测
    def _predict(self, sample, tree):
        if isinstance(tree, dict):  # 如果当前节点是字典，表示不是叶节点
            feature_index = tree['feature_index']
            threshold = tree['threshold']
            if sample[feature_index] < threshold:
                return self._predict(sample, tree['left'])
            else:
                return self._predict(sample, tree['right'])
        else:  # 如果是叶节点，返回类别
            return tree

    # 示例使用
if __name__ == "__main__":
    # 示例数据集
    X = np.array([[85, 85, 0],
                  [80, 70, 1],
                  [78, 90, 0],
                  [90, 95, 0],
                  [95, 60, 1],
                  [70, 70, 1],
                  [60, 80, 1],
                  [82, 80, 0],
                  [88, 70, 0],
                  [75, 60, 1]])
    y = np.array([0, 0, 1, 1, 1, 0, 0, 1, 1, 0])

    # 创建并训练决策树
    tree = DecisionTree(max_depth=3)
    tree.fit(X, y)

    # 进行预测
    predictions = tree.predict(X)
    print("预测结果:", predictions)


预测结果: [1, 1, 1, 1, 1, 0, 0, 1, 1, 0]
